In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

sf=pd.read_csv('/Users/dilrajkaur/Desktop/PANCAN_NETWORK/CESC/CESC-final.csv')

c1=pd.read_csv('/Users/dilrajkaur/Desktop/PANCAN_NETWORK/CESC/RF/hybrid.csv',header=None)

df0=sf[c1[0]]

/Users/dilrajkaur/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,8,9,19,64,89) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#df0.describe(include='all')
df0=df0.replace(to_replace='[Unknown]', value='Unknown', regex=False)
df0=df0.replace(to_replace='[NA]', value='Unknown', regex=False)
df0=df0.replace(to_replace='[Not Evaluated]', value='Unknown', regex=False)

In [3]:
#df0=df0.drop(df0.columns[df0.apply(lambda col:(col=='Unknown').sum()>(0.5*df0.shape[0]))], axis=1)
#df0.to_csv('/Users/macbook/Desktop/PANCAN-CLINICAL/dataset_reduced/BRCA-clinical.csv',index=False)

df=pd.DataFrame()
df=df0
#df=df0[['gender','age_at_diagnosis','tumor_status','margin_status','OS.time']]

df.loc[(df['OS.time']>df0['OS.time'].median()), 'OS.time'] = 'Low Risk'
df.loc[(df['OS.time']!='Low Risk'), 'OS.time'] = 'High Risk'

In [4]:
df

,OS.time,TRAF6,CD207,IRF4,KLRC2,LOXL2,TNFAIP3,ARRB2,NCAN,RIPK2,PGLYRP1,RNF125,NLRP1,SFTPA2,COLEC10,ajcc_tumor_pathologic_pt
0,High Risk,152.7182,6.4378,64.0200,0.3577,345.1323,293.2761,849.0701,1.7883,434.5494,0.0000,155.9371,443.1330,0.3577,0.3577,T2a2
1,High Risk,193.9342,54.6775,46.1341,6.4075,554.4255,1368.6459,500.2136,0.0000,298.1632,2.1358,58.5220,488.2529,2.5630,0.8543,T1b2
2,Low Risk,115.9249,140.2888,141.8607,67.1579,320.9981,2400.2358,791.8263,1.9648,345.8100,0.3930,94.3118,370.9598,0.0000,0.0000,T2b
3,Low Risk,178.1070,29.6271,140.2119,11.3685,3455.9642,1116.5274,886.0563,2.7560,324.1754,0.6890,38.5841,873.6543,1.0335,0.0000,T1b1
4,High Risk,131.0499,357.4088,134.6902,3.3921,290.5733,1393.8943,923.9679,0.6619,498.3867,0.9928,127.4096,274.6753,0.3309,0.9928,T1b1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,High Risk,149.8719,6.4048,89.6670,2.5619,1678.4458,3630.2306,421.4347,1.2810,432.5363,0.4270,49.1033,1152.8608,3.4159,0.4270,T3b
300,High Risk,140.5370,73.9145,695.0613,16.2413,511.7567,2969.1747,956.2479,0.6629,360.6231,0.0000,75.9032,498.5085,2.3202,1.6573,T2b
301,High Risk,101.3350,341.6262,418.6893,3.6408,407.0934,2305.2184,1039.4417,1.2136,744.5388,1.2136,72.8155,1635.3155,0.0000,0.6068,T3b
302,High Risk,193.0993,117.4334,18.1598,13.3293,2697.8874,1573.2446,780.8717,1.8160,386.1985,1.2107,17.5545,570.2179,0.6053,3.0266,T2b


In [5]:
cat_vars=['ajcc_tumor_pathologic_pt']

In [6]:
df_dummies=pd.get_dummies(df,columns=cat_vars)

In [13]:
df_dummies.shape

(304, 28)

In [12]:
df_dummies.drop(list(df_dummies.filter(regex = 'TX')), axis = 1, inplace = True)

In [10]:
df_dummies

,OS.time,TRAF6,CD207,IRF4,KLRC2,LOXL2,TNFAIP3,ARRB2,NCAN,RIPK2,...,ajcc_tumor_pathologic_pt_T2a,ajcc_tumor_pathologic_pt_T2a1,ajcc_tumor_pathologic_pt_T2a2,ajcc_tumor_pathologic_pt_T2b,ajcc_tumor_pathologic_pt_T3,ajcc_tumor_pathologic_pt_T3a,ajcc_tumor_pathologic_pt_T3b,ajcc_tumor_pathologic_pt_T4,ajcc_tumor_pathologic_pt_TX,ajcc_tumor_pathologic_pt_Tis
0,High Risk,152.7182,6.4378,64.0200,0.3577,345.1323,293.2761,849.0701,1.7883,434.5494,...,0,0,1,0,0,0,0,0,0,0
1,High Risk,193.9342,54.6775,46.1341,6.4075,554.4255,1368.6459,500.2136,0.0000,298.1632,...,0,0,0,0,0,0,0,0,0,0
2,Low Risk,115.9249,140.2888,141.8607,67.1579,320.9981,2400.2358,791.8263,1.9648,345.8100,...,0,0,0,1,0,0,0,0,0,0
3,Low Risk,178.1070,29.6271,140.2119,11.3685,3455.9642,1116.5274,886.0563,2.7560,324.1754,...,0,0,0,0,0,0,0,0,0,0
4,High Risk,131.0499,357.4088,134.6902,3.3921,290.5733,1393.8943,923.9679,0.6619,498.3867,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,High Risk,149.8719,6.4048,89.6670,2.5619,1678.4458,3630.2306,421.4347,1.2810,432.5363,...,0,0,0,0,0,0,1,0,0,0
300,High Risk,140.5370,73.9145,695.0613,16.2413,511.7567,2969.1747,956.2479,0.6629,360.6231,...,0,0,0,1,0,0,0,0,0,0
301,High Risk,101.3350,341.6262,418.6893,3.6408,407.0934,2305.2184,1039.4417,1.2136,744.5388,...,0,0,0,0,0,0,1,0,0,0
302,High Risk,193.0993,117.4334,18.1598,13.3293,2697.8874,1573.2446,780.8717,1.8160,386.1985,...,0,0,0,1,0,0,0,0,0,0


In [14]:
df_dummies.loc[(df_dummies['OS.time']=='Low Risk'), 'OS.time'] =  1
df_dummies.loc[(df_dummies['OS.time']=='High Risk'), 'OS.time'] =  0



#labels=df_dummies[df_dummies['OS.time']>]

In [15]:
labels=df_dummies[['OS.time']]

In [17]:
features=df_dummies.drop(['OS.time'],axis=1)

In [18]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.20,stratify=labels)

y_train=y_train.astype('int')
y_test=y_test.astype('int')

print(len(X_train),len(X_test))

243 61


In [19]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [37]:
pipe_steps=[('scaler',StandardScaler()),('RF',RandomForestClassifier())]

check_params={'RF__n_estimators':np.arange(100,200),
              'RF__criterion':['gini','entropy'],
             'RF__max_depth':np.arange(5,20)}

pipeline=Pipeline(pipe_steps)
print(pipeline)

Pipeline(steps=[('scaler', StandardScaler()), ('RF', RandomForestClassifier())])


In [38]:
import warnings
warnings.filterwarnings("ignore")

cv=5

create_grid=GridSearchCV(pipeline,param_grid=check_params,cv=cv)
create_grid.fit(features,labels_)

print("score for %d fold CV:=%3.2f" %(cv,create_grid.score(X_test,y_test)))
print("best params")
print(create_grid.best_params_)

score for 5 fold CV:=1.00
best params
{'RF__criterion': 'gini', 'RF__max_depth': 12, 'RF__n_estimators': 112}


In [39]:
#from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
#import pydotplus

RTree=RandomForestClassifier(criterion='entropy',max_depth=12,n_estimators=112)
RTree.fit(X_train,y_train)

# dot_data=StringIO()

# export_graphviz(DecsTree, out_file=dot_data,filled=True,rounded=True,special_characters=True,feature_names=features.columns,class_names=['High Risk','Low Risk'])

# graph=pydotplus.graph_from_dot_data(dot_data.getvalue())

# graph.write_png('BRCA_decs_tree.png',)
# Image(graph.create_png())

RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=112)

In [40]:
pred_train=RTree.predict_proba(X_train)[:,1]
actual_train=np.asarray(y_train.iloc[:,0])

pred_test=RTree.predict_proba(X_test)[:,1]
actual_test=np.asarray(y_test.iloc[:,0])


In [41]:
#low risk=0,high risk=1
def accuracy_decision(actual, pred, out,name,label):
    y_hat= pred#list(np.loadtxt(pred, delimiter=','))#output file--3
    y_actual= actual#list(np.loadtxt(actual, delimiter=','))#output file--3
    result1= open(out,'a', newline='')#output file--3
    with result1:
        writer1=csv.writer(result1);
        #writer1.writerow([name])
        writer1.writerow([name,'Threshold','TP', 'FP', 'TN', 'FN','sens','spec','acc','auroc','mcc','FPR']);
        for thr in np.arange(0, 1.1, 0.1):
            TP,FP,TN,FN,spec,mcc,sens,acc,i = 0,0,0,0,0,0,0,0,0
            #print (len(y_hat),len(y_actual))
            while i < len(y_hat):
                if y_actual[i]== 1 and y_hat[i] >= thr:
                    TP += 1
                elif y_actual[i]== 0 and y_hat[i] >= thr:
                    FP += 1
                elif y_actual[i] == 0 and y_hat[i] < thr:
                    TN += 1
                elif y_actual[i]== 1 and y_hat[i] < thr:
                    FN += 1
                i += 1          
            binder=TP+FN
            nonb=TN+FP
            total=TP+TN+FP+FN
            if binder!=0:
                sens=(TP/binder)*100
            else:
                sens == 0
            if nonb!=0:
                spec=TN/(nonb)*100
            else:
                spec == 0
            acc=((TP+TN)/total)*100
            FPR=100 - spec
            auc1=roc_auc_score(y_actual, y_hat)
            if ((TP+FN)*(TP+FP)*(FN+FP)*(TN+FN)) != 0:
                mcc=(TP*TN-FP*FN)/((TN+FN)*(TP+FN)*(TN+FP)*(TP+FP))**0.5
            else:
                mcc=0
            writer1=csv.writer(result1);
            writer1.writerow([label,thr,TP, FP, TN, FN,sens,spec,acc,auc1,mcc,FPR]);


In [42]:
import csv
from sklearn.metrics import roc_auc_score

accuracy_decision(actual_train,pred_train,'train_CESC.csv','train','train')
accuracy_decision(actual_test,pred_test,'test_CESC.csv','test','test')

In [43]:
np.asarray(y_train.iloc[:,0])

array([0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1])

In [44]:
#five fold accuracy
from sklearn.model_selection import cross_val_score

scores = cross_val_score(RTree, features, labels, cv=5)

In [45]:
labels_=labels.astype('float64')

In [46]:
scores

array([nan, nan, nan, nan, nan])